In [11]:
import numpy as np
import math
import time
from scipy.interpolate import CubicSpline # Pour l'interpolation Spline
from scipy.integrate import quad # Pour l'intégration numérique précise (pour tester Lagrange)

In [12]:
def rectangle_midpoint(f, a, b, n):
    if n <= 0: raise ValueError("n doit être positif.")
    h = (b - a) / n
    integral = sum(f(a + (i + 0.5) * h) for i in range(n)) * h
    return integral

def trapezoid(f, a, b, n):
    if n <= 0: raise ValueError("n doit être positif.")
    h = (b - a) / n
    integral = (f(a) + f(b)) / 2.0 + sum(f(a + i * h) for i in range(1, n))
    return integral * h

def simpson(f, a, b, n):
    if n <= 0: raise ValueError("n doit être positif.")
    if n % 2 != 0: raise ValueError("n pour Simpson doit être pair.")
    h = (b - a) / n
    integral = f(a) + f(b)
    for i in range(1, n):
        x_i = a + i * h
        integral += 4 * f(x_i) if i % 2 == 1 else 2 * f(x_i)
    return (h / 3) * integral

In [13]:
# --- Fonctions à intégrer ---
def f_x_squared(x):
    return x**2

def f_sin(x):
    return np.sin(x) # Correction ici, utilise np.sin

# Nouvelle fonction pour les tests de performance
def f_gaussian(x):
    return np.exp(-x**2)

In [14]:
def simpson_lagrange_analytic(f, a, b, n):
    """
    Calcule l'intégrale de f(x) de a à b en utilisant la méthode de Simpson,
    en construisant le polynôme de Lagrange de degré 2 sur chaque paire d'intervalles
    et en l'intégrant.

    Note: Pour cette implémentation, nous allons utiliser les coefficients (h/3, 4h/3, h/3)
    qui sont la conséquence directe de l'intégration analytique des polynômes de base de Lagrange.
    C'est la définition "analytique" de Simpson.
    
    Args:
        f (function): La fonction à intégrer.
        a (float): La borne inférieure de l'intervalle.
        b (float): La borne supérieure de l'intervalle.
        n (int): Le nombre de sous-intervalles (doit être pair).
        
    Returns:
        float: L'approximation de l'intégrale.
    """
    if n <= 0: raise ValueError("n doit être positif.")
    if n % 2 != 0: raise ValueError("n pour Simpson doit être pair.")

    h = (b - a) / n
    integral = 0.0

    # On parcourt les intervalles par paires (de x_0 à x_2, de x_2 à x_4, etc.)
    # Chaque segment de 2h est un "sous-intervalle" pour la formule de Simpson simple
    for i in range(0, n, 2): # i prend les valeurs 0, 2, 4, ..., n-2
        x0 = a + i * h
        x1 = a + (i + 1) * h
        x2 = a + (i + 2) * h # C'est le point x_{i+2}
        
        # Les coefficients de Simpson sont dérivés de l'intégration analytique
        # des polynômes de Lagrange de degré 2.
        # Nous appliquons directement ces coefficients.
        integral += (h / 3) * (f(x0) + 4 * f(x1) + f(x2))
        
    return integral

In [15]:
def simpson_via_spline_integration(f, a, b, n):
    """
    Calcule l'intégrale de f(x) de a à b en utilisant une interpolation par Spline Cubique Naturelle
    qui passe par les points (x_i, f(x_i)), puis intègre cette Spline.
    Ce n'est pas la méthode de Simpson à proprement parler, mais une intégration d'une fonction interpolée.
    
    Args:
        f (function): La fonction à interpoler et intégrer.
        a (float): La borne inférieure de l'intervalle.
        b (float): La borne supérieure de l'intervalle.
        n (int): Le nombre de points à utiliser pour l'interpolation (nombre de sous-intervalles).
                 Pour n sous-intervalles, il y a n+1 points.
        
    Returns:
        float: L'approximation de l'intégrale de la spline.
    """
    if n <= 0: raise ValueError("n doit être positif.")
    
    # Générer les points d'interpolation
    x_points = np.linspace(a, b, n + 1) # n sous-intervalles => n+1 points
    y_points = f(x_points) # Valeurs de la fonction aux points

    # Créer la Spline Cubique Naturelle
    # bc_type='natural' signifie que la dérivée seconde aux extrémités est zéro.
    spline = CubicSpline(x_points, y_points, bc_type='natural')

    # Intégrer la spline sur l'intervalle [a, b]
    # La méthode .integrate(xa, xb) de CubicSpline intègre la spline analytiquement.
    integral_spline = spline.integrate(a, b)
    
    return integral_spline

In [16]:
if __name__ == "__main__":
    # --- PARTIE 1 & 2 : Tests de précision (inchangés) ---
    a_test = 0
    b_test = 1
    n_test = 100 # Nombre de sous-intervalles
    
    print(f"--- Tests de précision (f(x) = x^2 sur [{a_test}, {b_test}], n={n_test}) ---")
    print(f"Intégrale exacte : 1/3 = {1/3:.6f}\n")

    rect_result = rectangle_midpoint(f_x_squared, a_test, b_test, n_test)
    print(f"Rectangles (point milieu): {rect_result:.6f}, Erreur: {abs(rect_result - (1/3)):.6f}")

    trap_result = trapezoid(f_x_squared, a_test, b_test, n_test)
    print(f"Trapèzes: {trap_result:.6f}, Erreur: {abs(trap_result - (1/3)):.6f}")

    simpson_result = simpson(f_x_squared, a_test, b_test, n_test)
    print(f"Simpson: {simpson_result:.6f}, Erreur: {abs(simpson_result - (1/3)):.6f}")

    simpson_lagrange_result = simpson_lagrange_analytic(f_x_squared, a_test, b_test, n_test)
    print(f"Simpson (Lagrange analytique): {simpson_lagrange_result:.6f}, Erreur: {abs(simpson_lagrange_result - (1/3)):.6f}")

    spline_result = simpson_via_spline_integration(f_x_squared, a_test, b_test, n_test)
    print(f"Intégration Spline Cubique: {spline_result:.6f}, Erreur: {abs(spline_result - (1/3)):.6f}\n")

    print("-" * 60)

    # --- PARTIE 3 : Comparaison de rapidité ---
    print("--- Comparaison de rapidité des méthodes d'intégration ---")

    a_perf = 0
    b_perf = 3
    # L'intégrale de e^(-x^2) n'a pas de forme fermée simple,
    # nous n'aurons donc pas une valeur exacte pour l'erreur, mais nous nous concentrons sur le temps.
    # Pour référence, l'intégrale de e^(-x^2) de 0 à infini est sqrt(pi)/2 approx 0.8862269
    # de 0 à 3, c'est environ 0.886207 (très proche de la valeur à l'infini)

    # Valeurs de n à tester
    n_values = [1000, 10000, 100000, 1000000] # Ajout de 1 million pour bien voir les différences

    methods = {
        "Rectangle (Midpoint)": rectangle_midpoint,
        "Trapèze": trapezoid,
        "Simpson": simpson,
        "Simpson (Lagrange Analytic)": simpson_lagrange_analytic,
        "Intégration Spline Cubique": simpson_via_spline_integration
    }

    results_perf = {}

    for name, method in methods.items():
        results_perf[name] = {}
        print(f"\nTesting method: {name}")
        for n_val in n_values:
            # Simpson et Simpson (Lagrange) nécessitent un n pair
            if (name == "Simpson" or name == "Simpson (Lagrange Analytic)") and n_val % 2 != 0:
                n_val += 1 # Assurez-vous que n est pair
                print(f"  Adjusting n to {n_val} for {name}")

            start_time = time.time()
            try:
                # Exécuter la méthode d'intégration
                # On utilise f_gaussian pour cette comparaison
                integral_approx = method(f_gaussian, a_perf, b_perf, n_val)
            except ValueError as e:
                integral_approx = f"ERROR: {e}"
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            results_perf[name][n_val] = {
                "integral": integral_approx,
                "time": elapsed_time
            }
            print(f"  n = {n_val:7d}: Time = {elapsed_time:.6f} seconds, Integral = {integral_approx if not isinstance(integral_approx, str) else 'Error'}")

    print("\n--- Synthèse des temps d'exécution ---")
    print(f"{'Method':<30} | {'n':<10} | {'Time (s)':<15} | {'Integral Approx':<20}")
    print("-" * 80)
    for name, data in results_perf.items():
        for n_val, res in data.items():
            integral_val = res["integral"]
            if isinstance(integral_val, float):
                integral_str = f"{integral_val:.8f}"
            else:
                integral_str = str(integral_val) # Afficher l'erreur si c'est une string
            print(f"{name:<30} | {n_val:<10} | {res['time']:<15.6f} | {integral_str:<20}")
    
    print("\nObservation: La méthode de Simpson (Lagrange Analytic) est conceptuellement la même que Simpson, elles ont donc une rapidité très similaire.")
    print("L'intégration Spline Cubique peut être plus lente pour un grand nombre de points car la construction de la spline elle-même prend du temps.")
    print("Les méthodes de Rectangle et Trapèze sont généralement les plus rapides par itération, mais Simpson et Spline peuvent atteindre une meilleure précision avec moins d'itérations.")

--- Tests de précision (f(x) = x^2 sur [0, 1], n=100) ---
Intégrale exacte : 1/3 = 0.333333

Rectangles (point milieu): 0.333325, Erreur: 0.000008
Trapèzes: 0.333350, Erreur: 0.000017
Simpson: 0.333333, Erreur: 0.000000
Simpson (Lagrange analytique): 0.333333, Erreur: 0.000000
Intégration Spline Cubique: 0.333333, Erreur: 0.000000

------------------------------------------------------------
--- Comparaison de rapidité des méthodes d'intégration ---

Testing method: Rectangle (Midpoint)
  n =    1000: Time = 0.028170 seconds, Integral = 0.8862073485371906
  n =   10000: Time = 0.369975 seconds, Integral = 0.886207348262296
  n =  100000: Time = 3.253321 seconds, Integral = 0.8862073482595618
  n = 1000000: Time = 36.869447 seconds, Integral = 0.8862073482595436

Testing method: Trapèze
  n =    1000: Time = 0.022882 seconds, Integral = 0.8862073477041791
  n =   10000: Time = 0.260072 seconds, Integral = 0.886207348253972
  n =  100000: Time = 2.224765 seconds, Integral = 0.88620734825